# Evaporation from KNMI data
*R.A. Collenteur, University of Graz, 2021*

In this notebook it is shown how to compute (potential) evaporation from meteorological data using PyEt. Meteorological data is observed by the KNMI at De Bilt in the Netherlands.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import pyet
pyet.show_versions()

## 1. Load KNMI Data

We first load the raw meteorological data observed by the KNMI at De Bilt in the Netherlands. This datafile contains a lot of different variables, please see the end of the notebook for an explanation of all the variables. 

In [ ]:
data = pd.read_csv("data/etmgeg_260.txt", skiprows=46, delimiter=",", 
                   skipinitialspace=True, index_col="YYYYMMDD", parse_dates=True).loc["2018",:]
data.head()

## 2. Estimating potential evaporation

Now that we have the input data, we can estimate potential evaporation with different estimation methods. Here we choose the Penman, Priestley-Taylor, Makkink, and Oudin methods. 

In [ ]:
# Preprocess the input data
meteo = pd.DataFrame({"tmean":data.TG/10, "tmax":data.TX/10, "tmin":data.TN/10, "rh":data.UG, "wind":data.FG/10, "rs":data.Q/100})
tmean, tmax, tmin, rh, wind, rs = [meteo[col] for col in meteo.columns]
pressure =  data.PG / 100  # to kPa
wind = data.FG/10  # to m/s
lat = 0.91  # Latitude of the meteorological station
elevation = 4  # meters above sea-level 

# Estimate evaporation with four different methods
pe_penman = pyet.penman(tmean, wind, rs=rs, elevation=4, lat=0.91, tmax=tmax, tmin=tmin, rh=rh)
pe_pt = pyet.priestley_taylor(tmean, wind, rs=rs, elevation=4, lat=0.91, tmax=tmax, tmin=tmin, rh=rh)
pe_makkink = pyet.makkink(tmean, rs, elevation=4, pressure=pressure)
pe_oudin = pyet.oudin(tmean, lat=0.91)

## 3. Plot the results

We plot the cumulative sums to compare the different estimation methods.

In [ ]:
fig, axs = plt.subplots(figsize=(12,3.5), ncols=2)
axs[0].plot(pe_penman,label="Penman")
axs[0].plot(pe_pt, label="Priestley-Taylor")
axs[0].plot(pe_makkink, label="Makkink - PyEt")
axs[0].plot(pe_oudin, label="Oudin")
axs[1].plot(pe_penman.cumsum(),label="Penman")
axs[1].plot(pe_pt.cumsum(), label="Priestley-Taylor")
axs[1].plot(pe_makkink.cumsum(), label="Makkink - PyEt")
axs[1].plot(pe_oudin.cumsum(), label="Oudin")
axs[0].set_ylabel("ET [mm/day]", fontsize=12)
axs[1].set_ylabel("Cumulative ET [mm]", fontsize=12)
for i in (0,1):
    axs[i].set_xlabel("Date", fontsize=12)
    axs[i].legend(loc=2)
plt.tight_layout()

#plt.savefig("Figure1.png", dpi=300)

## 4. Comparison: pyet Makkink vs KNMI Makkink

The KNMI also provides Makkink potential evaporation data (column EV24). We can now compare the results from Pyet and the KNMI to confirm that these are the same.

In [ ]:
pe_knmi = data.EV24 / 10  # Makkink potential evaporation computen by the KNMI for comparison

# Plot the two series against each other
plt.scatter(pe_makkink,pe_knmi)
plt.plot([0,6],[0,6], color="red", label="1:1 line")
plt.legend()
plt.xlabel("pyet-Makkink [mm]")
plt.ylabel("KNMI-Makkink [mm]");

## 5. Estimation of PE - all methods

In [ ]:
pe_hamon = pyet.hamon(tmean, 0.91)
pe_har = pyet.hargreaves(tmean, tmax, tmin, lat)
pe_jh1 = pyet.jensen_haise(tmean, rs=rs, method=1)
pe_jh2 = pyet.jensen_haise(tmean, lat=0.91, method=2)
pe_oudin = pyet.oudin(tmean, lat=0.91)
pe_abtew = pyet.abtew(tmean, rs)
pe_turc = pyet.turc(tmean, rs, rh)
pe_mg = pyet.mcguinness_bordne(tmean, lat=0.91)
pe_linacre = pyet.linacre(tmean, 4, 0.91, tmax=tmax, tmin=tmin)
pe_romanenko = pyet.romanenko(tmean, rh) 
pe_fao56 = pyet.pm_fao56(tmean, wind, rs=rs, elevation=4, lat=0.91, tmax=tmax, tmin=tmin, rh=rh)
pe_pm = pyet.pm(tmean, wind, rs=rs, elevation=4, lat=0.91, tmax=tmax, tmin=tmin, rh=rh)
pe_kb = pyet.kimberly_penman(tmean, wind, rs=rs, elevation=4, lat=0.91, tmax=tmax, tmin=tmin, rh=rh)
pe_fao24 = pyet.fao_24(tmean, wind, rs=rs, rh=rh, elevation=4)
pe_thom = pyet.thom_oliver(tmean, wind, rs=rs, elevation=4, lat=0.91, tmax=tmax, tmin=tmin, rh=rh)
pe_makkink = pyet.makkink(tmean, rs, elevation=4, pressure=pressure)

In [ ]:
pe_hamon.plot(label="Hamon")
pe_har.plot(label="Hargreaves")
pe_jh1.plot(label="Jensen-Haise 1")
pe_jh2.plot(label="Jensen-Haise 2")
pe_makkink.plot(label="Makkink")
pe_oudin.plot(label="Oudin")
pe_abtew.plot(label="Abtew")
pe_turc.plot(label="Turc")
pe_fao24.plot(label="FAO-24")
pe_mg.plot(label="McGuinness-Bordne")
plt.legend();

In [ ]:
pe_linacre.plot(label="Linacre")
pe_romanenko.plot(label="Romanenko")
pe_fao56.plot(label="FAO-56")
pe_pm.plot(label="Penman-Monteith")
pe_kb.plot(label="Kimberley-Penman")
pe_fao24.plot(label="FAO-24")
pe_thom.plot(label="Thon and Oliver")
plt.legend();

## References

Column description of the KNMI data

- DDVEC     = Vector mean wind direction in degrees (360=north, 90=east, 180=south, 270=west, 0=calm/variable)
- FHVEC     = Vector mean windspeed (in 0.1 m/s)
- FG        = Daily mean windspeed (in 0.1 m/s) 
- FHX       = Maximum hourly mean windspeed (in 0.1 m/s)
- FHXH      = Hourly division in which FHX was measured
- FHN       = Minimum hourly mean windspeed (in 0.1 m/s)
- FHNH      = Hourly division in which FHN was measured
- FXX       = Maximum wind gust (in 0.1 m/s)
- FXXH      = Hourly division in which FXX was measured
- TG        = Daily mean temperature in (0.1 degrees Celsius)
- TN        = Minimum temperature (in 0.1 degrees Celsius)
- TNH       = Hourly division in which TN was measured
- TX        = Maximum temperature (in 0.1 degrees Celsius)
- TXH       = Hourly division in which TX was measured
- T10N      = Minimum temperature at 10 cm above surface (in 0.1 degrees Celsius)
- T10NH     = 6-hourly division in which T10N was measured; 6=0-6 UT, 12=6-12 UT, 18=12-18 UT, 24=18-24 UT 
- SQ        = Sunshine duration (in 0.1 hour) calculated from global radiation (-1 for <0.05 hour)
- SP        = Percentage of maximum potential sunshine duration
- Q         = Global radiation (in J/cm2)
- DR        = Precipitation duration (in 0.1 hour)
- RH        = Daily precipitation amount (in 0.1 mm) (-1 for <0.05 mm)
- RHX       = Maximum hourly precipitation amount (in 0.1 mm) (-1 for <0.05 mm)
- RHXH      = Hourly division in which RHX was measured
- PG        = Daily mean sea level pressure (in 0.1 hPa) calculated from 24 hourly values
- PX        = Maximum hourly sea level pressure (in 0.1 hPa)
- PXH       = Hourly division in which PX was measured
- PN        = Minimum hourly sea level pressure (in 0.1 hPa)
- PNH       = Hourly division in which PN was measured
- VVN       = Minimum visibility; 0: <100 m, 1:100-200 m, 2:200-300 m,..., 49:4900-5000 m, 50:5-6 km, 56:6-7 km, 57:7-8 km,..., 79:29-30 km, 80:30-35 km, 81:35-40 km,..., 89: >70 km)
- VVNH      = Hourly division in which VVN was measured
- VVX       = Maximum visibility; 0: <100 m, 1:100-200 m, 2:200-300 m,..., 49:4900-5000 m, 50:5-6 km, 56:6-7 km, 57:7-8 km,..., 79:29-30 km, 80:30-35 km, 81:35-40 km,..., 89: >70 km)
- VVXH      = Hourly division in which VVX was measured
- NG        = Mean daily cloud cover (in octants, 9=sky invisible)
- UG        = Daily mean relative atmospheric humidity (in percents)
- UX        = Maximum relative atmospheric humidity (in percents)
- UXH       = Hourly division in which UX was measured
- UN        = Minimum relative atmospheric humidity (in percents)
- UNH       = Hourly division in which UN was measured
- EV24      = Potential evapotranspiration (Makkink) (in 0.1 mm)